In [1]:
import dgl
import math
import torch
import numpy as np
import networkx as nx
from os import path
from pathlib import Path
from copy import deepcopy
from dgl.data import DGLDataset
from ToyDGLDataset import ToyDGLDataset
from GraphDatasetInfo import (DistributionType, Distribution, GraphSubdatasetInfo, GraphDatasetInfo)


Using backend: pytorch


In [2]:
# default stuff
toyname = 'Toy0'
rootOutputFolder = path.join('/home/andrew/GNN_Sandbox/GraphToyDatasets', toyname)
graphCountLarge = 2000
graphCountSmall = 100

fewNodesPerGraph0 = Distribution(
    distributionType=DistributionType.truncnorm, 
    minimum=1, maximum=10, 
    mean=6, standardDeviation=1)

fewNodesPerGraph1 = Distribution(
    distributionType=DistributionType.truncnorm, 
    minimum=5, maximum=20, 
    mean=13, standardDeviation=1)

fewNodesPerGraphSameDist = Distribution(
    distributionType=DistributionType.truncnorm, 
    minimum=1, maximum=20, 
    mean=10, standardDeviation=1)

manyNodesPerGraph0 = Distribution(
    distributionType=DistributionType.truncnorm, 
    minimum=10, maximum=120, 
    mean=30, standardDeviation=10)

manyNodesPerGraph1 = Distribution(
    distributionType=DistributionType.truncnorm, 
    minimum=10, maximum=120, 
    mean=65, standardDeviation=10)

manyNodesPerGraphSameDist = Distribution(
    distributionType=DistributionType.truncnorm, 
    minimum=10, maximum=120, 
    mean=65, standardDeviation=10)

nFeatMapping = {'P_t': 0, 'Eta': 1, 'Phi': 2, 'Mass': 3, 'Type': 4}

defaultNodeFeat = [
    Distribution(10, 100, DistributionType.uniform), # index 0 -> P_t
    Distribution(-10, 10, DistributionType.uniform), # index 1 -> Eta
    Distribution(0, 2 * math.pi, DistributionType.uniform), # index 2 -> Phi
    Distribution(0.001, 1, DistributionType.uniform), # index 3 -> Mass
    Distribution(0, 2, DistributionType.uniform, roundToNearestInt=True) # index 4 -> Type
]

eFeatMapping = {'DeltaEta': 0, 'DeltaPhi': 1, 'RapiditySquared': 2}

edgeFeat = [
    None, # index 0 -> DeltaEta
    None, # index 1 -> DeltaPhi
    None # index 2 -> RapiditySquared
]

graphlabel = [0, 1]
splitPercentages = {'train': 0.6, 'valid': 0.2, 'test': 0.2}

In [3]:
datasetname = toyname + '_' + 'fewGraphs_fewNodes_SameNodeCountDistributions'
outputFolder = path.join(rootOutputFolder, datasetname)
Path(outputFolder).mkdir(parents=True, exist_ok=True)

name = f'GraphClass{graphlabel[0]}'

graphsubdatasetInfo0 = GraphSubdatasetInfo(
    name=name, label=graphlabel[0], 
    graphCount=graphCountSmall, nodesPerGraph=fewNodesPerGraphSameDist, 
    nodeFeatMapping=nFeatMapping, nodeFeat=defaultNodeFeat, 
    edgeFeatMapping=eFeatMapping, edgeFeat=edgeFeat)

name = f'GraphClass{graphlabel[1]}'

graphsubdatasetInfo1 = deepcopy(graphsubdatasetInfo0)
graphsubdatasetInfo1.name = name
graphsubdatasetInfo1.label = graphlabel[1]
#graphsubdatasetInfo1.nodeFeat[nFeatMapping['P_t']] = Distribution(60, 80, DistributionType.uniform)

subdatasets = []
subdatasets.append(graphsubdatasetInfo1)
subdatasets.append(graphsubdatasetInfo0)

graphdatasetInfo = GraphDatasetInfo(
    name=datasetname,
    splitPercentages=splitPercentages,
    graphSubDatasetInfos=subdatasets
)

graphdatasetInfo.SaveToJsonfile(outputFolder, f'{graphdatasetInfo.name}.json')
dataset = ToyDGLDataset(name=datasetname, info=graphdatasetInfo, shuffleDataset=True, save_dir=outputFolder)

#can be loaded from json file like below:
#graphdatasetInfo = GraphDatasetInfo.LoadFromJsonfile(path.join(outputFolder, f'{graphdatasetInfo.name}.json'))

File Toy0_fewGraphs_fewNodes_SameNodeCountDistributions.json already exists! Do you want to overwrite the file? (y/n)
y


(1/2) Generating graphs from SubDataset GraphClass1: 100%|██████████| 100/100 [00:00<00:00, 2028.37it/s]
(2/2) Generating graphs from SubDataset GraphClass0: 100%|██████████| 100/100 [00:00<00:00, 2150.30it/s]

File Toy0_fewGraphs_fewNodes_SameNodeCountDistributions.json overwritten.
Calculating and saving histograms...
[100, 10, 6, 1, 2]
[10, -10, 0, 0, 0]
[20, 6, 419]
[0, 0, 0]
{'P_t': array([ 10.        ,  14.73684211,  19.47368421,  24.21052632,
        28.94736842,  33.68421053,  38.42105263,  43.15789474,
        47.89473684,  52.63157895,  57.36842105,  62.10526316,
        66.84210526,  71.57894737,  76.31578947,  81.05263158,
        85.78947368,  90.52631579,  95.26315789, 100.        ]), 'Eta': array([-10.        ,  -8.94736842,  -7.89473684,  -6.84210526,
        -5.78947368,  -4.73684211,  -3.68421053,  -2.63157895,
        -1.57894737,  -0.52631579,   0.52631579,   1.57894737,
         2.63157895,   3.68421053,   4.73684211,   5.78947368,
         6.84210526,   7.89473684,   8.94736842,  10.        ]), 'Phi': array([0.        , 0.31578947, 0.63157895, 0.94736842, 1.26315789,
       1.57894737, 1.89473684, 2.21052632, 2.52631579, 2.84210526,
       3.15789474, 3.47368421, 3.78947

Num Graph classes: 2
Graph classes: [0, 1]
Number of graphs: 200
Number of all nodes in all graphs: 1903
Number of all edges in all graphs: 16394
Dim node features: 5
Node feature keys: ['P_t', 'Eta', 'Phi', 'Mass', 'Type']
Dim edge features: 3
Edge feature keys: ['DeltaEta', 'DeltaPhi', 'RapiditySquared']
Done saving data into cached files.


<Figure size 720x504 with 0 Axes>

In [4]:
datasetname = toyname + '_' + 'fewGraphs_fewNodes_DifferentNodeCountDistributions'
outputFolder = path.join(rootOutputFolder, datasetname)
Path(outputFolder).mkdir(parents=True, exist_ok=True)

name = f'GraphClass{graphlabel[0]}'

graphsubdatasetInfo0 = GraphSubdatasetInfo(
    name=name, label=graphlabel[0], 
    graphCount=graphCountSmall, nodesPerGraph=fewNodesPerGraph0, 
    nodeFeatMapping=nFeatMapping, nodeFeat=defaultNodeFeat, 
    edgeFeatMapping=eFeatMapping, edgeFeat=edgeFeat)

name = f'GraphClass{graphlabel[1]}'

graphsubdatasetInfo1 = deepcopy(graphsubdatasetInfo0)
graphsubdatasetInfo1.name = name
graphsubdatasetInfo1.label = graphlabel[1]
graphsubdatasetInfo1.nodesPerGraph = fewNodesPerGraph1
#graphsubdatasetInfo1.nodeFeat[nFeatMapping['P_t']] = Distribution(60, 80, DistributionType.uniform)

subdatasets = []
subdatasets.append(graphsubdatasetInfo1)
subdatasets.append(graphsubdatasetInfo0)

graphdatasetInfo = GraphDatasetInfo(
    name=datasetname,
    splitPercentages=splitPercentages,
    graphSubDatasetInfos=subdatasets
)

graphdatasetInfo.SaveToJsonfile(outputFolder, f'{graphdatasetInfo.name}.json')
dataset = ToyDGLDataset(name=datasetname, info=graphdatasetInfo, shuffleDataset=True, save_dir=outputFolder)

(1/2) Generating graphs from SubDataset GraphClass1: 100%|██████████| 100/100 [00:00<00:00, 1722.37it/s]
(2/2) Generating graphs from SubDataset GraphClass0: 100%|██████████| 100/100 [00:00<00:00, 2712.79it/s]

Calculating and saving histograms...
[100, 10, 6, 1, 2]
[10, -10, 0, 0, 0]
[20, 6, 405]
[0, 0, 0]
{'P_t': array([ 10.        ,  14.73684211,  19.47368421,  24.21052632,
        28.94736842,  33.68421053,  38.42105263,  43.15789474,
        47.89473684,  52.63157895,  57.36842105,  62.10526316,
        66.84210526,  71.57894737,  76.31578947,  81.05263158,
        85.78947368,  90.52631579,  95.26315789, 100.        ]), 'Eta': array([-10.        ,  -8.94736842,  -7.89473684,  -6.84210526,
        -5.78947368,  -4.73684211,  -3.68421053,  -2.63157895,
        -1.57894737,  -0.52631579,   0.52631579,   1.57894737,
         2.63157895,   3.68421053,   4.73684211,   5.78947368,
         6.84210526,   7.89473684,   8.94736842,  10.        ]), 'Phi': array([0.        , 0.31578947, 0.63157895, 0.94736842, 1.26315789,
       1.57894737, 1.89473684, 2.21052632, 2.52631579, 2.84210526,
       3.15789474, 3.47368421, 3.78947368, 4.10526316, 4.42105263,
       4.73684211, 5.05263158, 5.36842105, 5.

Num Graph classes: 2
Graph classes: [0, 1]
Number of graphs: 200
Number of all nodes in all graphs: 1819
Number of all edges in all graphs: 17408
Dim node features: 5
Node feature keys: ['P_t', 'Eta', 'Phi', 'Mass', 'Type']
Dim edge features: 3
Edge feature keys: ['DeltaEta', 'DeltaPhi', 'RapiditySquared']
Done saving data into cached files.


<Figure size 720x504 with 0 Axes>

In [5]:
datasetname = toyname + '_' + 'fewGraphs_manyNodes_SameNodeCountDistributions'
outputFolder = path.join(rootOutputFolder, datasetname)
Path(outputFolder).mkdir(parents=True, exist_ok=True)

name = f'GraphClass{graphlabel[0]}'

graphsubdatasetInfo0 = GraphSubdatasetInfo(
    name=name, label=graphlabel[0], 
    graphCount=graphCountSmall, nodesPerGraph=manyNodesPerGraphSameDist, 
    nodeFeatMapping=nFeatMapping, nodeFeat=defaultNodeFeat, 
    edgeFeatMapping=eFeatMapping, edgeFeat=edgeFeat)

name = f'GraphClass{graphlabel[1]}'

graphsubdatasetInfo1 = deepcopy(graphsubdatasetInfo0)
graphsubdatasetInfo1.name = name
graphsubdatasetInfo1.label = graphlabel[1]
#graphsubdatasetInfo1.nodeFeat[nFeatMapping['P_t']] = Distribution(60, 80, DistributionType.uniform)

subdatasets = []
subdatasets.append(graphsubdatasetInfo1)
subdatasets.append(graphsubdatasetInfo0)

graphdatasetInfo = GraphDatasetInfo(
    name=datasetname,
    splitPercentages=splitPercentages,
    graphSubDatasetInfos=subdatasets
)

graphdatasetInfo.SaveToJsonfile(outputFolder, f'{graphdatasetInfo.name}.json')
dataset = ToyDGLDataset(name=datasetname, info=graphdatasetInfo, shuffleDataset=True, save_dir=outputFolder)

(1/2) Generating graphs from SubDataset GraphClass1: 100%|██████████| 100/100 [00:00<00:00, 113.24it/s]
(2/2) Generating graphs from SubDataset GraphClass0: 100%|██████████| 100/100 [00:00<00:00, 112.52it/s]


Calculating and saving histograms...
[100, 10, 6, 1, 2]
[10, -10, 0, 0, 0]
[20, 6, 421]
[0, 0, 0]
{'P_t': array([ 10.        ,  14.73684211,  19.47368421,  24.21052632,
        28.94736842,  33.68421053,  38.42105263,  43.15789474,
        47.89473684,  52.63157895,  57.36842105,  62.10526316,
        66.84210526,  71.57894737,  76.31578947,  81.05263158,
        85.78947368,  90.52631579,  95.26315789, 100.        ]), 'Eta': array([-10.        ,  -8.94736842,  -7.89473684,  -6.84210526,
        -5.78947368,  -4.73684211,  -3.68421053,  -2.63157895,
        -1.57894737,  -0.52631579,   0.52631579,   1.57894737,
         2.63157895,   3.68421053,   4.73684211,   5.78947368,
         6.84210526,   7.89473684,   8.94736842,  10.        ]), 'Phi': array([0.        , 0.31578947, 0.63157895, 0.94736842, 1.26315789,
       1.57894737, 1.89473684, 2.21052632, 2.52631579, 2.84210526,
       3.15789474, 3.47368421, 3.78947368, 4.10526316, 4.42105263,
       4.73684211, 5.05263158, 5.36842105, 5.

<Figure size 720x504 with 0 Axes>

In [6]:
datasetname = toyname + '_' + 'fewGraphs_manyNodes_DifferentNodeCountDistributions'
outputFolder = path.join(rootOutputFolder, datasetname)
Path(outputFolder).mkdir(parents=True, exist_ok=True)

name = f'GraphClass{graphlabel[0]}'

graphsubdatasetInfo0 = GraphSubdatasetInfo(
    name=name, label=graphlabel[0], 
    graphCount=graphCountSmall, nodesPerGraph=manyNodesPerGraph0, 
    nodeFeatMapping=nFeatMapping, nodeFeat=defaultNodeFeat, 
    edgeFeatMapping=eFeatMapping, edgeFeat=edgeFeat)

name = f'GraphClass{graphlabel[1]}'

graphsubdatasetInfo1 = deepcopy(graphsubdatasetInfo0)
graphsubdatasetInfo1.name = name
graphsubdatasetInfo1.label = graphlabel[1]
graphsubdatasetInfo1.nodesPerGraph = manyNodesPerGraph1
#graphsubdatasetInfo1.nodeFeat[nFeatMapping['P_t']] = Distribution(60, 80, DistributionType.uniform)

subdatasets = []
subdatasets.append(graphsubdatasetInfo1)
subdatasets.append(graphsubdatasetInfo0)

graphdatasetInfo = GraphDatasetInfo(
    name=datasetname,
    splitPercentages=splitPercentages,
    graphSubDatasetInfos=subdatasets
)

graphdatasetInfo.SaveToJsonfile(outputFolder, f'{graphdatasetInfo.name}.json')
dataset = ToyDGLDataset(name=datasetname, info=graphdatasetInfo, shuffleDataset=True, save_dir=outputFolder)

(1/2) Generating graphs from SubDataset GraphClass1: 100%|██████████| 100/100 [00:00<00:00, 130.51it/s]
(2/2) Generating graphs from SubDataset GraphClass0: 100%|██████████| 100/100 [00:00<00:00, 558.75it/s]


Calculating and saving histograms...
[100, 10, 6, 1, 2]
[10, -10, 0, 0, 0]
[20, 6, 420]
[0, 0, 0]
{'P_t': array([ 10.        ,  14.73684211,  19.47368421,  24.21052632,
        28.94736842,  33.68421053,  38.42105263,  43.15789474,
        47.89473684,  52.63157895,  57.36842105,  62.10526316,
        66.84210526,  71.57894737,  76.31578947,  81.05263158,
        85.78947368,  90.52631579,  95.26315789, 100.        ]), 'Eta': array([-10.        ,  -8.94736842,  -7.89473684,  -6.84210526,
        -5.78947368,  -4.73684211,  -3.68421053,  -2.63157895,
        -1.57894737,  -0.52631579,   0.52631579,   1.57894737,
         2.63157895,   3.68421053,   4.73684211,   5.78947368,
         6.84210526,   7.89473684,   8.94736842,  10.        ]), 'Phi': array([0.        , 0.31578947, 0.63157895, 0.94736842, 1.26315789,
       1.57894737, 1.89473684, 2.21052632, 2.52631579, 2.84210526,
       3.15789474, 3.47368421, 3.78947368, 4.10526316, 4.42105263,
       4.73684211, 5.05263158, 5.36842105, 5.

<Figure size 720x504 with 0 Axes>

In [7]:
datasetname = toyname + '_' + 'manyGraphs_fewNodes_SameNodeCountDistributions'
outputFolder = path.join(rootOutputFolder, datasetname)
Path(outputFolder).mkdir(parents=True, exist_ok=True)

name = f'GraphClass{graphlabel[0]}'

graphsubdatasetInfo0 = GraphSubdatasetInfo(
    name=name, label=graphlabel[0], 
    graphCount=graphCountLarge, nodesPerGraph=fewNodesPerGraphSameDist, 
    nodeFeatMapping=nFeatMapping, nodeFeat=defaultNodeFeat, 
    edgeFeatMapping=eFeatMapping, edgeFeat=edgeFeat)

name = f'GraphClass{graphlabel[1]}'

graphsubdatasetInfo1 = deepcopy(graphsubdatasetInfo0)
graphsubdatasetInfo1.name = name
graphsubdatasetInfo1.label = graphlabel[1]
#graphsubdatasetInfo1.nodeFeat[nFeatMapping['P_t']] = Distribution(60, 80, DistributionType.uniform)

subdatasets = []
subdatasets.append(graphsubdatasetInfo1)
subdatasets.append(graphsubdatasetInfo0)

graphdatasetInfo = GraphDatasetInfo(
    name=datasetname,
    splitPercentages=splitPercentages,
    graphSubDatasetInfos=subdatasets
)

graphdatasetInfo.SaveToJsonfile(outputFolder, f'{graphdatasetInfo.name}.json')
dataset = ToyDGLDataset(name=datasetname, info=graphdatasetInfo, shuffleDataset=True, save_dir=outputFolder)

(1/2) Generating graphs from SubDataset GraphClass1: 100%|██████████| 2000/2000 [00:00<00:00, 2168.37it/s]
(2/2) Generating graphs from SubDataset GraphClass0: 100%|██████████| 2000/2000 [00:00<00:00, 2152.11it/s]


Calculating and saving histograms...
[100, 10, 6, 1, 2]
[10, -10, 0, 0, 0]
[20, 6, 417]
[0, 0, 0]
{'P_t': array([ 10.        ,  14.73684211,  19.47368421,  24.21052632,
        28.94736842,  33.68421053,  38.42105263,  43.15789474,
        47.89473684,  52.63157895,  57.36842105,  62.10526316,
        66.84210526,  71.57894737,  76.31578947,  81.05263158,
        85.78947368,  90.52631579,  95.26315789, 100.        ]), 'Eta': array([-10.        ,  -8.94736842,  -7.89473684,  -6.84210526,
        -5.78947368,  -4.73684211,  -3.68421053,  -2.63157895,
        -1.57894737,  -0.52631579,   0.52631579,   1.57894737,
         2.63157895,   3.68421053,   4.73684211,   5.78947368,
         6.84210526,   7.89473684,   8.94736842,  10.        ]), 'Phi': array([0.        , 0.31578947, 0.63157895, 0.94736842, 1.26315789,
       1.57894737, 1.89473684, 2.21052632, 2.52631579, 2.84210526,
       3.15789474, 3.47368421, 3.78947368, 4.10526316, 4.42105263,
       4.73684211, 5.05263158, 5.36842105, 5.

<Figure size 720x504 with 0 Axes>

In [8]:
datasetname = toyname + '_' + 'manyGraphs_fewNodes_DifferentNodeCountDistributions'
outputFolder = path.join(rootOutputFolder, datasetname)
Path(outputFolder).mkdir(parents=True, exist_ok=True)

name = f'GraphClass{graphlabel[0]}'

graphsubdatasetInfo0 = GraphSubdatasetInfo(
    name=name, label=graphlabel[0], 
    graphCount=graphCountLarge, nodesPerGraph=fewNodesPerGraph0, 
    nodeFeatMapping=nFeatMapping, nodeFeat=defaultNodeFeat, 
    edgeFeatMapping=eFeatMapping, edgeFeat=edgeFeat)

name = f'GraphClass{graphlabel[1]}'

graphsubdatasetInfo1 = deepcopy(graphsubdatasetInfo0)
graphsubdatasetInfo1.name = name
graphsubdatasetInfo1.label = graphlabel[1]
graphsubdatasetInfo1.nodesPerGraph = fewNodesPerGraph1
#graphsubdatasetInfo1.nodeFeat[nFeatMapping['P_t']] = Distribution(60, 80, DistributionType.uniform)

subdatasets = []
subdatasets.append(graphsubdatasetInfo1)
subdatasets.append(graphsubdatasetInfo0)

graphdatasetInfo = GraphDatasetInfo(
    name=datasetname,
    splitPercentages=splitPercentages,
    graphSubDatasetInfos=subdatasets
)

graphdatasetInfo.SaveToJsonfile(outputFolder, f'{graphdatasetInfo.name}.json')
dataset = ToyDGLDataset(name=datasetname, info=graphdatasetInfo, shuffleDataset=True, save_dir=outputFolder)

(1/2) Generating graphs from SubDataset GraphClass1: 100%|██████████| 2000/2000 [00:01<00:00, 1722.82it/s]
(2/2) Generating graphs from SubDataset GraphClass0: 100%|██████████| 2000/2000 [00:00<00:00, 2825.56it/s]


Calculating and saving histograms...
[100, 10, 6, 1, 2]
[10, -10, 0, 0, 0]
[20, 6, 420]
[0, 0, 0]
{'P_t': array([ 10.        ,  14.73684211,  19.47368421,  24.21052632,
        28.94736842,  33.68421053,  38.42105263,  43.15789474,
        47.89473684,  52.63157895,  57.36842105,  62.10526316,
        66.84210526,  71.57894737,  76.31578947,  81.05263158,
        85.78947368,  90.52631579,  95.26315789, 100.        ]), 'Eta': array([-10.        ,  -8.94736842,  -7.89473684,  -6.84210526,
        -5.78947368,  -4.73684211,  -3.68421053,  -2.63157895,
        -1.57894737,  -0.52631579,   0.52631579,   1.57894737,
         2.63157895,   3.68421053,   4.73684211,   5.78947368,
         6.84210526,   7.89473684,   8.94736842,  10.        ]), 'Phi': array([0.        , 0.31578947, 0.63157895, 0.94736842, 1.26315789,
       1.57894737, 1.89473684, 2.21052632, 2.52631579, 2.84210526,
       3.15789474, 3.47368421, 3.78947368, 4.10526316, 4.42105263,
       4.73684211, 5.05263158, 5.36842105, 5.

<Figure size 720x504 with 0 Axes>

In [9]:
datasetname = toyname + '_' + 'manyGraphs_manyNodes_SameNodeCountDistributions'
outputFolder = path.join(rootOutputFolder, datasetname)
Path(outputFolder).mkdir(parents=True, exist_ok=True)

name = f'GraphClass{graphlabel[0]}'

graphsubdatasetInfo0 = GraphSubdatasetInfo(
    name=name, label=graphlabel[0], 
    graphCount=graphCountLarge, nodesPerGraph=manyNodesPerGraphSameDist, 
    nodeFeatMapping=nFeatMapping, nodeFeat=defaultNodeFeat, 
    edgeFeatMapping=eFeatMapping, edgeFeat=edgeFeat)

name = f'GraphClass{graphlabel[1]}'

graphsubdatasetInfo1 = deepcopy(graphsubdatasetInfo0)
graphsubdatasetInfo1.name = name
graphsubdatasetInfo1.label = graphlabel[1]
#graphsubdatasetInfo1.nodeFeat[nFeatMapping['P_t']] = Distribution(60, 80, DistributionType.uniform)

subdatasets = []
subdatasets.append(graphsubdatasetInfo1)
subdatasets.append(graphsubdatasetInfo0)

graphdatasetInfo = GraphDatasetInfo(
    name=datasetname,
    splitPercentages=splitPercentages,
    graphSubDatasetInfos=subdatasets
)

graphdatasetInfo.SaveToJsonfile(outputFolder, f'{graphdatasetInfo.name}.json')
dataset = ToyDGLDataset(name=datasetname, info=graphdatasetInfo, shuffleDataset=True, save_dir=outputFolder)

(1/2) Generating graphs from SubDataset GraphClass1: 100%|██████████| 2000/2000 [00:15<00:00, 126.88it/s]
(2/2) Generating graphs from SubDataset GraphClass0: 100%|██████████| 2000/2000 [00:15<00:00, 132.68it/s]


Calculating and saving histograms...
[100, 10, 6, 1, 2]
[10, -10, 0, 0, 0]
[20, 6, 433]
[0, 0, 0]
{'P_t': array([ 10.        ,  14.73684211,  19.47368421,  24.21052632,
        28.94736842,  33.68421053,  38.42105263,  43.15789474,
        47.89473684,  52.63157895,  57.36842105,  62.10526316,
        66.84210526,  71.57894737,  76.31578947,  81.05263158,
        85.78947368,  90.52631579,  95.26315789, 100.        ]), 'Eta': array([-10.        ,  -8.94736842,  -7.89473684,  -6.84210526,
        -5.78947368,  -4.73684211,  -3.68421053,  -2.63157895,
        -1.57894737,  -0.52631579,   0.52631579,   1.57894737,
         2.63157895,   3.68421053,   4.73684211,   5.78947368,
         6.84210526,   7.89473684,   8.94736842,  10.        ]), 'Phi': array([0.        , 0.31578947, 0.63157895, 0.94736842, 1.26315789,
       1.57894737, 1.89473684, 2.21052632, 2.52631579, 2.84210526,
       3.15789474, 3.47368421, 3.78947368, 4.10526316, 4.42105263,
       4.73684211, 5.05263158, 5.36842105, 5.

<Figure size 720x504 with 0 Axes>

In [10]:
datasetname = toyname + '_' + 'manyGraphs_manyNodes_DifferentNodeCountDistributions'
outputFolder = path.join(rootOutputFolder, datasetname)
Path(outputFolder).mkdir(parents=True, exist_ok=True)

name = f'GraphClass{graphlabel[0]}'

graphsubdatasetInfo0 = GraphSubdatasetInfo(
    name=name, label=graphlabel[0], 
    graphCount=graphCountLarge, nodesPerGraph=manyNodesPerGraph0, 
    nodeFeatMapping=nFeatMapping, nodeFeat=defaultNodeFeat, 
    edgeFeatMapping=eFeatMapping, edgeFeat=edgeFeat)

name = f'GraphClass{graphlabel[1]}'

graphsubdatasetInfo1 = deepcopy(graphsubdatasetInfo0)
graphsubdatasetInfo1.name = name
graphsubdatasetInfo1.label = graphlabel[1]
graphsubdatasetInfo1.nodesPerGraph = manyNodesPerGraph1
#graphsubdatasetInfo1.nodeFeat[nFeatMapping['P_t']] = Distribution(60, 80, DistributionType.uniform)

subdatasets = []
subdatasets.append(graphsubdatasetInfo1)
subdatasets.append(graphsubdatasetInfo0)

graphdatasetInfo = GraphDatasetInfo(
    name=datasetname,
    splitPercentages=splitPercentages,
    graphSubDatasetInfos=subdatasets
)

graphdatasetInfo.SaveToJsonfile(outputFolder, f'{graphdatasetInfo.name}.json')
dataset = ToyDGLDataset(name=datasetname, info=graphdatasetInfo, shuffleDataset=True, save_dir=outputFolder)

(1/2) Generating graphs from SubDataset GraphClass1: 100%|██████████| 2000/2000 [00:15<00:00, 127.69it/s]
(2/2) Generating graphs from SubDataset GraphClass0: 100%|██████████| 2000/2000 [00:03<00:00, 504.80it/s]


Calculating and saving histograms...
[100, 10, 6, 1, 2]
[10, -10, 0, 0, 0]
[20, 6, 431]
[0, 0, 0]
{'P_t': array([ 10.        ,  14.73684211,  19.47368421,  24.21052632,
        28.94736842,  33.68421053,  38.42105263,  43.15789474,
        47.89473684,  52.63157895,  57.36842105,  62.10526316,
        66.84210526,  71.57894737,  76.31578947,  81.05263158,
        85.78947368,  90.52631579,  95.26315789, 100.        ]), 'Eta': array([-10.        ,  -8.94736842,  -7.89473684,  -6.84210526,
        -5.78947368,  -4.73684211,  -3.68421053,  -2.63157895,
        -1.57894737,  -0.52631579,   0.52631579,   1.57894737,
         2.63157895,   3.68421053,   4.73684211,   5.78947368,
         6.84210526,   7.89473684,   8.94736842,  10.        ]), 'Phi': array([0.        , 0.31578947, 0.63157895, 0.94736842, 1.26315789,
       1.57894737, 1.89473684, 2.21052632, 2.52631579, 2.84210526,
       3.15789474, 3.47368421, 3.78947368, 4.10526316, 4.42105263,
       4.73684211, 5.05263158, 5.36842105, 5.

<Figure size 720x504 with 0 Axes>

In [11]:
rng = np.random.default_rng(seed=42)
nxgraphs = graphdatasetInfo.ToNetworkxGraphList()

print(f'Edge Features \n nx: {list(nxgraphs[0].edges.data())[0]},\n dgl: {dglgraph.edata}')

print(f'Graphs in the dataset: {len(nxgraphs)}')
print('Node features of the first graph in the graph list: ')
for node in nxgraphs[0].nodes(data=True):
    print(node)

print("Fully connected graph with edge features: ")
print(nxgraphs[0].edges.data())

pos = nx.spring_layout(nxgraphs[0])
options = {
    "node_color": "#A0CBE2",
    "width": 0.5,
    "with_labels": True,
    "node_size": 600
}
plt.figure(1,figsize=(10,10)) 
nx.draw(nxgraphs[0], pos, **options)

dglgraph = dgl.from_networkx(
    nxgraphs[0], 
    node_attrs=nodeFeatures.keys(), 
    edge_attrs=edgeFeatures.keys())
print(f'Node count - nx: {nxgraphs[0].number_of_nodes()}, dgl: {dglgraph.num_nodes()}')
print(f'Edge count - nx: {nxgraphs[0].number_of_edges()}, dgl: {dglgraph.num_edges()}')

print(f'Node Features \n nx: {nxgraphs[0].nodes(data=True)[0]},\n dgl: {dglgraph.ndata}')

print(dglgraph)

AttributeError: 'GraphDatasetInfo' object has no attribute 'ToNetworkxGraphList'